In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import copy
import pyautogui as pta

In [2]:
pta.FAILSAFE = False

In [3]:
fmask = cv2.createBackgroundSubtractorMOG2(detectShadows=False)

In [4]:
def get_thresh(frame):
    img_changed = fmask.apply(frame)
    res = cv2.bitwise_and(frame,frame,mask=img_changed)
    gray_res = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
    blur_res = cv2.GaussianBlur(gray_res,(41,41),0)
    _,thresh = cv2.threshold(blur_res,60,255,cv2.THRESH_BINARY)
    return thresh

In [24]:
def get_point(thresh):
    thresh1 = copy.deepcopy(thresh)
    conto , hier = cv2.findContours(thresh1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    fin_img = np.zeros(thresh1.shape,np.uint8)
    try:
        max_cont = max(conto,key= lambda x: cv2.contourArea(x))
    except:
        return None,None,None
    hull = cv2.convexHull(max_cont)
    try:
        topmost = tuple(max_cont[max_cont[:,:,1].argmin()][0])
    except:
        return None,None,None
    cv2.circle(fin_img,topmost,8,(124,69,45),-1)
    return fin_img,topmost,max_cont

In [17]:
cap = cv2.VideoCapture(0)
while True:
    ret,frame = cap.read()
    thresh = get_thresh(frame)
    cv2.imshow('frame',frame)
    cv2.imshow('thresh',thresh)
    if cv2.waitKey(10)==ord('s'):
        while True:
            ret,frame = cap.read()
            thresh = get_thresh(frame)
            fin_image,topmost = get_point(thresh)
            if fin_image is not None:
                cv2.imshow('fin_img',fin_image)
                pta.moveTo(1920-3*topmost[0],3*topmost[1],0)
            if cv2.waitKey(10)==ord('b'):
                break
    if cv2.waitKey(10)==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [12]:
thresh.shape

(480, 640)

In [22]:
cv2.imshow('image',thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()